In [ ]:
import os
import pandas as pd
from utils import partial_match_scores


# root = "../datasets/myriadlama/llama3.2_1b_it"
# root = "../datasets/myriadlama/llama3.2_3b_it"
# root = "../datasets/myriadlama/llama3.1_8b_it"
# root = "../datasets/myriadlama/qwen2.5_3b_it"
# root = "../datasets/myriadlama/qwen2.5_7b_it"

# 使用你的qwen1.5_moe_a2.7b_chat模型结果
root = "/net/tokyo100-10g/data/str01_01/y-guo/datasets/myriadlama/qwen1.5_moe_a2.7b_chat"

# root = "./datasets/webqa/llama3.2_1b_it"
idx = 10

In [ ]:
# 检查可用的文件
import os
print("Available files in the directory:")
for file in os.listdir(root):
    print(f"  {file}")

# 如果没有confidence.feather，我们将直接分析ensemble结果
confidence_file = os.path.join(root, "confidence.feather")
if os.path.exists(confidence_file):
    print(f"\nFound confidence.feather, analyzing individual paraphrase performance...")
    from utils import partial_match
    
    df = pd.read_feather(confidence_file)
    df["sample_lemmas"] = df["sample_lemmas"].apply(lambda xs: [list(x) for x in xs])
    df["answer_lemmas"] = df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])

    all_matches = []
    all_predictions = []
    all_answers = []
    for uuid, sdf in df.groupby("uuid"):
        all_predictions.append(sdf['greedy_lemma'].tolist())
        all_answers.append(sdf['answer_lemmas'].tolist()[0])

    all_accs = []
    for idx, predictions in enumerate(zip(*all_predictions)):
        matches = []
        for prediction, _gold_answers in zip(predictions, all_answers):
            score = partial_match(prediction, _gold_answers, False)
            matches.append(int(score))
        
        all_matches.append(matches)
        print(f"Paraphrase {idx}: {sum(matches)/len(matches):.3f}")
        all_accs.append(sum(matches)/len(matches))

    print(f"Average paraphrase accuracy: {sum(all_accs)/len(all_accs):.3f}")
else:
    print(f"\nconfidence.feather not found. Skipping individual paraphrase analysis.")
    print("To generate this file, run: python confidence.py --model qwen1.5_moe_a2.7b_chat")

Paraphrase 0: 0.447
Paraphrase 1: 0.370
Paraphrase 2: 0.486
Paraphrase 3: 0.453
Paraphrase 4: 0.459
Paraphrase 5: 0.398
Paraphrase 6: 0.393
Paraphrase 7: 0.394
Paraphrase 8: 0.391
Paraphrase 9: 0.398
Average paraphrase accuracy: 0.419


In [3]:
df[df["uuid"] == list(df["uuid"].unique())[1]]

,uuid,paraphrase,prompt,answers,greedy_predict,sample_predicts,greedy_lemma,sample_lemmas,answer_lemmas,confidence
1,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert has work experience in...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, philosophy, philosophy, philosop...",[mathematic],"[[mathematic], [philosophy], [philosophy], [ph...","[[physics], [physical], [physic], [general, ph...",0.65
9,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert works in the field of ...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, mathematics, mathematics, mathem...",[mathematic],"[[mathematic], [mathematic], [mathematic], [ma...","[[physics], [physical], [physic], [general, ph...",0.95
17,fd915def246a0be-1df91599c979dd7,Which field does Johann Heinrich Lambert work ...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, mathematics, mathematics, mathem...",[mathematic],"[[mathematic], [mathematic], [mathematic], [ma...","[[physics], [physical], [physic], [general, ph...",0.87
25,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert is influential in the ...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, philosophy, mathematics, philoso...",[mathematic],"[[mathematic], [philosophy], [mathematic], [ph...","[[physics], [physical], [physic], [general, ph...",0.75
33,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert made significant contr...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, geometry, geometry, geometry, ma...",[mathematic],"[[mathematic], [geometry], [geometry], [geomet...","[[physics], [physical], [physic], [general, ph...",0.59
41,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert has taken up a career ...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",philosophy,"[mathematics, philosophy, philosophy, philosop...",[philosophy],"[[mathematic], [philosophy], [philosophy], [ph...","[[physics], [physical], [physic], [general, ph...",0.42
49,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert is a major contributor...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",philosophy,"[philosophy, philosophy, philosophy, philosoph...",[philosophy],"[[philosophy], [philosophy], [philosophy], [ph...","[[physics], [physical], [physic], [general, ph...",1.00
57,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert is seasoned in the pra...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, philosophy, philosophy, philosop...",[mathematic],"[[mathematic], [philosophy], [philosophy], [ph...","[[physics], [physical], [physic], [general, ph...",0.69
65,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert is famous for the adva...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",geometry,"[geometry, geometry, geometry, geometry, geome...",[geometry],"[[geometry], [geometry], [geometry], [geometry...","[[physics], [physical], [physic], [general, ph...",0.97
73,fd915def246a0be-1df91599c979dd7,Johann Heinrich Lambert has a career in what f...,Predict the [MASK] in the sentence in one word...,"[physics, physical, physic, general physics, p...",mathematics,"[mathematics, mathematics, mathematics, mathem...",[mathematic],"[[mathematic], [mathematic], [mathematic], [ma...","[[physics], [physical], [physic], [general, ph...",0.94


In [ ]:
def get_ensemble_scores(path):
    if not os.path.exists(path):
        return -1
    df = pd.read_feather(path)
    df["answer_lemmas"] = df["answer_lemmas"].apply(lambda xs: [list(x) for x in xs])

    answers = [ans for ans in df["answer_lemmas"].tolist()]
    
    avg_acc = partial_match_scores(df['predict_lemma'].tolist(), answers)
    return avg_acc

print("=== Ensemble Performance Analysis ===")
print("Available ensemble results:")

# 检查所有可能的ensemble文件
ensemble_files = []
for idx in range(2, 11):
    for method in ['avg', 'max', 'weighted_avg', 'weighted_max']:
        filename = f"ensemble_{method}-{idx}.feather"
        filepath = os.path.join(root, filename)
        if os.path.exists(filepath):
            ensemble_files.append((method, idx, filepath))

if not ensemble_files:
    print("No ensemble files found.")
else:
    print(f"Found {len(ensemble_files)} ensemble result files:")
    
    results = {}
    for method, idx, filepath in ensemble_files:
        acc = get_ensemble_scores(filepath)
        if method not in results:
            results[method] = {}
        results[method][idx] = acc
        print(f"  Ensemble {method}-{idx}: {acc:.3f}")
    
    print(f"\n=== Summary ===")
    for method in ['avg', 'max', 'weighted_avg', 'weighted_max']:
        if method in results:
            best_idx = max(results[method].keys(), key=lambda k: results[method][k])
            best_acc = results[method][best_idx]
            print(f"Best {method}: ensemble-{best_idx} with {best_acc:.3f} accuracy")

Ensemble 2: avg=0.452, max=0.451, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 3: avg=0.489, max=0.473, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 4: avg=0.500, max=0.476, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 5: avg=0.502, max=0.481, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 6: avg=0.508, max=0.474, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 7: avg=0.509, max=0.468, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 8: avg=0.514, max=0.468, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 9: avg=0.511, max=0.461, weighted_avg=-1.000, weighted_max=-1.000
Ensemble 10: avg=0.503, max=0.459, weighted_avg=-1.000, weighted_max=-1.000


In [5]:
for i in range(2, 11):
    or_match = sum(any(ms) for ms in zip(*all_matches[:i]))
    print(f"Ratio of or_match with {i} paraphrases: {or_match / len(all_matches[0]):.3f}")

    and_match = sum(all(ms) for ms in zip(*all_matches[:i]))
    print(f"Ratio of and_match with {i} paraphrases: {and_match / len(all_matches[0]):.3f}\n")
    

Ratio of or_match with 2 paraphrases: 0.503
Ratio of and_match with 2 paraphrases: 0.314

Ratio of or_match with 3 paraphrases: 0.568
Ratio of and_match with 3 paraphrases: 0.283

Ratio of or_match with 4 paraphrases: 0.590
Ratio of and_match with 4 paraphrases: 0.267

Ratio of or_match with 5 paraphrases: 0.607
Ratio of and_match with 5 paraphrases: 0.242

Ratio of or_match with 6 paraphrases: 0.617
Ratio of and_match with 6 paraphrases: 0.206

Ratio of or_match with 7 paraphrases: 0.625
Ratio of and_match with 7 paraphrases: 0.181

Ratio of or_match with 8 paraphrases: 0.632
Ratio of and_match with 8 paraphrases: 0.160

Ratio of or_match with 9 paraphrases: 0.640
Ratio of and_match with 9 paraphrases: 0.141

Ratio of or_match with 10 paraphrases: 0.646
Ratio of and_match with 10 paraphrases: 0.128

